<a href="https://colab.research.google.com/github/sherryfish321/urban-expansion-probability/blob/main/GeoProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install geopandas rasterio shapely pyproj scikit-learn folium mapclassify rtree


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 28.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/data/
!ls -al


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/data
total 396125
-rw------- 1 root root    930974 Jul 18  2022 gadm41_BRA_2.dbf
-rw------- 1 root root       145 Jul 18  2022 gadm41_BRA_2.prj
-rw------- 1 root root 263959916 Jul 18  2022 gadm41_BRA_2.shp
-rw------- 1 root root     44676 Jul 18  2022 gadm41_BRA_2.shx
-rw------- 1 root root         5 Aug 28 22:09 rio_boundary.cpg
-rw------- 1 root root      1491 Aug 28 22:09 rio_boundary.dbf
-rw------- 1 root root       145 Aug 28 22:09 rio_boundary.prj
-rw------- 1 root root    469432 Aug 28 22:09 rio_boundary.shp
-rw------- 1 root root       108 Aug 28 22:09 rio_boundary.shx
-rw------- 1 root root  71584733 Aug 26 07:48 worldcover_2020.tif
-rw------- 1 root root  68637369 Aug 26 07:53 worldcover_2021.tif


In [ ]:
import geopandas as gpd
from pathlib import Path

DATA_DIR = Path("/content/drive/MyDrive/Colab Notebooks/data/")

admin = gpd.read_file(DATA_DIR/"gadm41_BRA_2.shp")
admin.head()


,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,BRA.1.1_2,BRA,Brazil,BRA.1_1,Acre,NA,AcrelÃ¢ndia,NA,NA,MunicÃ­pio,Municipality,1200013,NA,"MULTIPOLYGON (((-67.00087 -10.22907, -67.00163..."
1,BRA.1.2_2,BRA,Brazil,BRA.1_1,Acre,NA,Assis Brasil,NA,NA,MunicÃ­pio,Municipality,1200054,NA,"POLYGON ((-69.57415 -10.94523, -69.57451 -10.9..."
2,BRA.1.3_2,BRA,Brazil,BRA.1_1,Acre,NA,BrasilÃ©ia,NA,NA,MunicÃ­pio,Municipality,1200104,NA,"POLYGON ((-69.41283 -10.93708, -69.41289 -10.9..."
3,BRA.1.4_2,BRA,Brazil,BRA.1_1,Acre,NA,Bujari,NA,NA,MunicÃ­pio,Municipality,1200138,NA,"POLYGON ((-67.97684 -9.35273, -67.97684 -9.352..."
4,BRA.1.5_2,BRA,Brazil,BRA.1_1,Acre,NA,Capixaba,NA,NA,MunicÃ­pio,Municipality,1200179,NA,"MULTIPOLYGON (((-67.57861 -10.50274, -67.57927..."


In [ ]:
# Filter the Rio de Janeiro city boundary (GADM level 2)
rio = admin[admin["NAME_2"]=="Rio de Janeiro"].copy()
assert len(rio) > 0, "找不到 Rio de Janeiro（請檢查 GADM 欄位名稱或改用 NAME_2 的實際值）"

# Save as an Independent Shapefile(for future use)
rio_path = DATA_DIR/"rio_boundary.shp"
rio.to_file(rio_path)
rio


,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
3660,BRA.19.68_2,BRA,Brazil,BRA.19_1,Rio de Janeiro,NA,Rio de Janeiro,NA,NA,MunicÃ­pio,Municipality,3304557,NA,"MULTIPOLYGON (((-43.55997 -23.08009, -43.55984..."


In [ ]:
import rasterio
from rasterio.mask import mask

lc2020_path = DATA_DIR/"worldcover_2020.tif"
lc2021_path = DATA_DIR/"worldcover_2021.tif"

# Read 2020/2021 imagery
r2020 = rasterio.open(lc2020_path)
r2021 = rasterio.open(lc2021_path)

# Project the AOI to match the imagery (to avoid coordinate system inconsistency)
rio_in_raster_crs = rio.to_crs(r2020.crs)
geoms = [rio_in_raster_crs.geometry.unary_union.__geo_interface__]

# Clip the two-period imagery using the AOI (to significantly reduce subsequent computational load)
arr2020, transform2020 = mask(r2020, geoms, crop=True)
arr2021, transform2021 = mask(r2021, geoms, crop=True)

arr2020 = arr2020[0]  # Single band
arr2021 = arr2021[0]
arr2020.shape, arr2021.shape


/tmp/ipython-input-2125303813.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geoms = [rio_in_raster_crs.geometry.unary_union.__geo_interface__]


((4037, 8366), (4037, 8366))

In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Point, box
import shapely
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Create a regular grid (using the pixel centers of the clipped imagery as points)
height, width = arr2020.shape
transform = transform2020  # The two periods are clipped to the same AOI, with consistent transform.

# Sampling interval: select one point for every N pixels, with N adjustable (larger N → fewer points → faster processing)
STEP = 5

rows = np.arange(0, height, STEP)
cols = np.arange(0, width, STEP)

# Convert row–column indices to coordinates (pixel centers)
def rowcol_to_xy(row, col, transform):
    x, y = rasterio.transform.xy(transform, row, col, offset='center')
    return x, y

coords = [rowcol_to_xy(r, c, transform) for r in rows for c in cols]
gdf_points = gpd.GeoDataFrame(geometry=[Point(xy) for xy in coords], crs=r2020.crs)

# Retain only points within the AOI
gdf_points = gdf_points[gdf_points.within(rio_in_raster_crs.geometry.unary_union)].copy()
len(gdf_points)


/tmp/ipython-input-3470043945.py:27: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gdf_points = gdf_points[gdf_points.within(rio_in_raster_crs.geometry.unary_union)].copy()


608632

In [ ]:
# Safe read function: return np.nan when out of bounds or encountering NoData
def sample_at_rowcol(arr, row, col):
    if 0 <= row < arr.shape[0] and 0 <= col < arr.shape[1]:
        return arr[row, col]
    return np.nan

# Inverse mapping to row, col (simplest: using the affine transform)
def xy_to_rowcol(x, y, transform):
    col, row = ~transform * (x, y)  # Affine inversion
    return int(round(row)), int(round(col))

lc20_vals, lc21_vals, rows_list, cols_list = [], [], [], []
for geom in gdf_points.geometry:
    x, y = geom.x, geom.y
    row, col = xy_to_rowcol(x, y, transform)
    rows_list.append(row); cols_list.append(col)
    lc20_vals.append(sample_at_rowcol(arr2020, row, col))
    lc21_vals.append(sample_at_rowcol(arr2021, row, col))

gdf_points["lc2020"] = lc20_vals
gdf_points["lc2021"] = lc21_vals

# Remove invalid pixels
gdf_points = gdf_points.dropna(subset=["lc2020", "lc2021"]).copy()
gdf_points.head()


,geometry,lc2020,lc2021
3276,POINT (-43.12862 -22.74646),80,10
3277,POINT (-43.12821 -22.74646),10,30
3278,POINT (-43.12779 -22.74646),10,10
3279,POINT (-43.12738 -22.74646),10,10
3280,POINT (-43.12696 -22.74646),30,10


In [12]:
BUILT = 50
gdf_points["target"] = ((gdf_points["lc2020"] != BUILT) & (gdf_points["lc2021"] == BUILT)).astype(int)
gdf_points["target"].value_counts()


,count
target,
0,590400
1,18232


In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

# City center point (geometric centroid of the AOI)
city_center = rio_in_raster_crs.geometry.unary_union.centroid
gdf_points["dist_center"] = gdf_points.distance(city_center)

# One-hot on lc2020
enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
X_cat = enc.fit_transform(gdf_points[["lc2020"]])

# Integrate features
X = np.concatenate([X_cat, gdf_points[["dist_center"]].to_numpy()], axis=1)
y = gdf_points["target"].to_numpy()

# Train/validation split (handling class imbalance with class_weight='balanced')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

clf = LogisticRegression(max_iter=2000, class_weight="balanced", n_jobs=-1)
clf.fit(X_train, y_train)

y_prob = clf.predict_proba(X_test)[:,1]
y_pred = (y_prob >= 0.5).astype(int)

print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print(classification_report(y_test, y_pred, digits=4))


/tmp/ipython-input-659012323.py:7: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  city_center = rio_in_raster_crs.geometry.unary_union.centroid


ROC-AUC: 0.7974923012649958
              precision    recall  f1-score   support

           0     0.9976    0.5178    0.6817    118081
           1     0.0579    0.9602    0.1093      3646

    accuracy                         0.5310    121727
   macro avg     0.5278    0.7390    0.3955    121727
weighted avg     0.9695    0.5310    0.6646    121727



In [14]:
import folium
import numpy as np
from branca.colormap import linear
import os

# Make predictions for all points
all_prob = clf.predict_proba(X)[:,1]
gdf_points["prob"] = all_prob

# Reproject to WGS84 for visualization in Folium
gdf_wgs84 = gdf_points.to_crs(epsg=4326)
rio_wgs84 = rio_in_raster_crs.to_crs(epsg=4326)

# Center of the map
cent = rio_wgs84.geometry.unary_union.centroid
m = folium.Map(location=[cent.y, cent.x], zoom_start=10, tiles="CartoDB positron")

# AOI boundary
folium.GeoJson(rio_wgs84.__geo_interface__, name="AOI").add_to(m)

# 色帶
cmap = linear.YlOrRd_09.scale(gdf_wgs84["prob"].min(), gdf_wgs84["prob"].max())

# Plot points with CircleMarker (if too many points, display a sample or switch to an isopleth/heatmap)
for _, row in gdf_wgs84.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=cmap(row["prob"])
    ).add_to(m)

cmap.caption = 'Urbanization Probability'
cmap.add_to(m)

os.makedirs("results", exist_ok=True)
m.save("results/risk_map.html")
"Saved to results/risk_map.html"


/tmp/ipython-input-2944068234.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  cent = rio_wgs84.geometry.unary_union.centroid


'Saved to results/risk_map.html'

In [15]:
out_csv = "results/predictions_points.csv"
gdf_wgs84.assign(lon=gdf_wgs84.geometry.x, lat=gdf_wgs84.geometry.y) \
         .drop(columns="geometry").to_csv(out_csv, index=False)
out_csv


'results/predictions_points.csv'

In [16]:
from google.colab import files
files.download("results/risk_map.html")
files.download("results/predictions_points.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>